<a href="https://colab.research.google.com/github/Nokondi/AI-ML-Training-Code/blob/main/AI-ML/Dijkstra's_Algorithm_Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly.graph_objects as go
import pandas as pd
from geopy import distance

State capitols originally from: https://github.com/jasperdebie/VisInfo/blob/master/us-state-capitals.csv

In [2]:
url = "https://drive.google.com/uc?export=download&id=1zg4A9kv-FtaHnX7aSxeoAkIUcUHrSTvh"
capitols = pd.read_csv(url)
capitols.head()

,state,capitol,latitude,longitude
0,AL,Montgomery,32.377716,-86.300568
1,AK,Juneau,58.301598,-134.420212
2,AZ,Phoenix,33.448143,-112.096962
3,AR,Little Rock,34.746613,-92.288986
4,CA,Sacramento,38.576668,-121.493629


Neighboring States originally from: https://github.com/ubikuity/List-of-neighboring-states-for-each-US-state/blob/master/neighbors-states.csv

In [3]:
url2 = "https://drive.google.com/uc?export=download&id=1izqLvH3vREJn-0k0slarIDQpJFcPngSR"
neighbor_states = pd.read_csv(url2)
neighbor_states.head()

,StateCode,NeighborStateCode
0,AK,WA
1,AL,FL
2,AL,GA
3,AL,MS
4,AL,TN


In [4]:
neighbors = {}
for index, row in capitols.iterrows():
    latlon = (float(row['latitude']), float(row['longitude']))
    n = {}
    ns = list(neighbor_states.loc[neighbor_states["StateCode"] == row['state']]["NeighborStateCode"])
    ns += list(neighbor_states.loc[neighbor_states["NeighborStateCode"] == row['state']]["StateCode"])
    for x in ns:
        dat = capitols.loc[capitols['state'] == x]
        if not dat.empty:
            cap = dat['capitol'].values[0]
            latlon2 = (float(dat['latitude'].values[0]), float(dat['longitude'].values[0]))
            dist = distance.distance(latlon, latlon2).miles
            n[cap] = dist
    neighbors[row['capitol']] = n

print(neighbors)

{'Montgomery': {'Tallahassee': 179.1376629135767, 'Atlanta': 145.74952594935303, 'Jackson': 227.10333904994764, 'Nashville': 262.56390741077314}, 'Juneau': {'Olympia': 914.3040227770455}, 'Phoenix': {'Sacramento': 633.5602777014598, 'Denver': 586.5729645109296, 'Santa Fe': 383.28814023065894, 'Carson City': 581.3841258250087, 'Salt Lake City': 505.5610048631618}, 'Little Rock': {'Baton Rouge': 302.48240775175975, 'Jefferson City': 264.3526100526838, 'Jackson': 207.6701458474815, 'Oklahoma City': 299.74864976363125, 'Nashville': 325.49972458438117, 'Austin': 442.86607500812994}, 'Sacramento': {'Honolulu': 2461.7061854861327, 'Carson City': 101.58029572843422, 'Salem': 446.1578559059242, 'Phoenix': 633.5602777014598}, 'Denver': {'Topeka': 500.22252431625407, 'Lincoln': 443.8533730636241, 'Santa Fe': 284.64058020602994, 'Oklahoma City': 503.92816781608866, 'Salt Lake City': 371.77690185651744, 'Cheyenne': 97.0585129597313, 'Phoenix': 586.5729645109296}, 'Hartford': {'Boston': 92.791585466

In [6]:
def Dijkstra(graph, start, goal):
    # Steps 1 and 2: Initialize our 3 lists and add the starting node to seen and parents
    visited = {}
    seen = {start: 0}
    parents = {start: None}

    while seen:
        # Step 3: retrieve the shortest distance node, add it to visited and remove it from seen
        currentDistance = sorted(seen.values())[0]
        current = [node for node in seen.keys() if seen[node] == currentDistance][0]
        visited[current] = currentDistance
        seen.pop(current)

        # Step 4: if we have reached the goal, end the loop
        if current == goal:
            break

        # Step 5: expand all nodes adjacent to the current node
        for neighbor, distance in graph[current].items():
            if neighbor in seen.keys(): 
                newDistance = currentDistance + distance
                if newDistance < seen[neighbor]:
                    seen[neighbor] = newDistance
                    parents[neighbor] = current
            elif neighbor not in visited.keys():
                seen[neighbor] = currentDistance + distance
                parents[neighbor] = current
        #Step 6: if seen is empty, exit loop; else, continue

    # Step 7: backtrack through parents to find the path, then reverse it
    path = [goal]
    node = goal
    while node != start:
        path.append(parents[node])
        node = parents[node]
    path.reverse()

    # Return the path and total distance
    return path, visited[goal]

In [7]:
origin = 'Sacramento'
destination = 'Augusta'
path, distance = Dijkstra(neighbors, origin, destination)

In [8]:
lats = []
lons = []
for city in path:
    lat = float(capitols.loc[capitols['capitol'] == city]['latitude'])
    lon = float(capitols.loc[capitols['capitol'] == city]['longitude'])
    lats.append(lat)
    lons.append(lon)

In [9]:
fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = lons,
    lat = lats,
    marker = {'size': 10}))

fig.add_trace(go.Scattermapbox(
    mode="markers+text",
    lon = capitols['longitude'],
    lat = capitols['latitude'],
    text = capitols['capitol'] + ", " + capitols['state'],
    marker = {'size': 10}
))

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 10, 'lat': 10},
        'style': "stamen-terrain",
        'center': {'lon': -20, 'lat': -20},
        'zoom': 1})

fig.show()